In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [52]:
%aimport src.config
%aimport src.helpers

In [17]:
import numpy as np
from pathlib import Path
from time import time
import json
import pandas as pd
from xml.etree.ElementTree import iterparse
from datetime import datetime

In [53]:
from src.config import data_dir
from src.helpers import calc_metrics

#### Process raw SMS data

In [51]:
filename = "karim-sms-allow.xml"
source = data_dir / filename
data = []
for event, elem in iterparse(source):
    if elem.tag == "sms":
        #if any(elem.attrib["body"]==r["text"] for r in data):
        #    continue
        record = {}
        record["text"] = elem.attrib["body"]
        record["contact_name"] = elem.attrib["contact_name"]
        record["address"] = elem.attrib["address"]
        record["timestamp"] = int(elem.attrib["date"])
        record["type"] = elem.attrib["type"]
        data.append(record)

In [55]:
df = pd.DataFrame(data)
df.to_excel(data_dir / "karim-sms-allow.xlsx", index=False)

#### Read labeled data

In [46]:
labeled_filename = "karim-sms-allow-labeled.xlsx"
labeled = pd.read_excel(data_dir / labeled_filename, sheet_name="total sms")
labeled["timestamp"] = (labeled["timestamp"] / 1000).map(datetime.fromtimestamp)
labeled["resp"] = 0

In [41]:
mapp = {"ham": 0, "spam": 1}

In [47]:
responses_filename = "SMS Data Collection (Responses).xlsx"
responses = pd.read_excel(data_dir / responses_filename)
responses = responses.rename(columns={"SMS text": "text", 
                                      "Is it a spam or ham?": "label",
                                     "Timestamp": "timestamp"})
responses["resp"] = 1
responses["label"] = responses["label"].map(lambda x: mapp.get(x, x))

In [54]:
total = pd.concat([labeled, responses], ignore_index=True)
total.to_excel(data_dir / "sms-uk-total.xlsx")

In [51]:
# Check dimensionality and class imbalance
total.shape
total.label.value_counts(normalize=True).round(5)*100
labeled.tail()

(3497, 8)

0    78.81
1    21.19
Name: label, dtype: float64

,address,contact_name,text,timestamp,type,label,service,resp
3223,+380631576335,Dad,I can't talk right now. I'll call you back sho...,2018-04-27 17:45:49.700,2,0,1,0
3224,+380937808130,Танька,Я вже на зв’язку з 28.04 02:40. Передзвоніть ...,2018-04-28 02:40:12.545,1,0,1,0
3225,lifecell,(Unknown),Я намагався Вам додзвонитися. Передзвоніть мен...,2018-04-28 12:55:47.947,1,0,1,0
3226,+380638197928,Mom,"У Вас пропущених дзвінків: 1 від мене, останн...",2018-04-28 14:41:39.942,1,0,1,0
3227,+380442251208,(Unknown),"У Вас пропущених дзвінків: 1 від мене, останн...",2018-04-30 09:27:35.264,1,0,1,0
